---
Author:                 **`Crispen Gari`**

Topic:                  **`"Named Entity Recognition" (NER)`**
 
Main:                   **`Natural Language Processing NLP`**

Library:                **`TensorFlow (2.x)`**

Programing Language:    **`Python`**

Date:                   **`2021-09-20`**

---


### NER (Named Entity Recognition) using transformers.

In this notebook we are going to do NER using transformers. Basically we will be following [this](https://keras.io/examples/nlp/ner_transformers/) amazing tensorflow tutorial from keras examples.

### Definition

Named Entity Recognition (NER) is the process of identifying named entities in text. Example of named entities are: "Person", "Location", "Organization", "Dates" etc. NER is essentially a token classification task where every token is classified into one or more predetermined categories.

The dataset that we will be working with is `HuggingFace` dataset so to download it we will run the following command.


In [1]:
!pip install datasets

We aslso need the script that will evaluate our NER models, so to download it we will run the following command.

In [ ]:
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py

### Imports

In [3]:
import os, time

import numpy as np
from tensorflow import keras
import tensorflow as tf
from collections import Counter
from conlleval import evaluate
from datasets import load_dataset

### Building a TransformerBlock layer

We are going to then build the Transformer block layer using keras SubClassing API in the following cell

In [26]:
class TransformerBlock(keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads, ff_dim, rate=.1):
    super(TransformerBlock, self).__init__()

    self.att = keras.layers.MultiHeadAttention(
        num_heads, key_dim = embedding_dim
    )
    self.ffn = keras.Sequential([
        keras.layers.Dense(ff_dim, activation="relu"),
        keras.layers.Dense(embedding_dim)
    ])
    self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
    self.dropout1 = keras.layers.Dropout(rate)
    self.dropout2 = keras.layers.Dropout(rate)

  def call(self, inputs, training=False):
    attn_output = self.att(inputs, inputs)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(inputs + attn_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    return self.layernorm2(out1 + ffn_output)

### Building the `TokenAndPositionEmbedding`

In [27]:
class TokenAndPositionEmbedding(keras.layers.Layer):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super(TokenAndPositionEmbedding, self).__init__()

    self.token_emb = keras.layers.Embedding(
        input_dim=vocab_size, output_dim=embed_dim
    )
    self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

  def call(self, inputs):
    maxlen = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=maxlen, delta=1)
    position_embeddings = self.pos_emb(positions)
    token_embeddings = self.token_emb(inputs)
    return token_embeddings + position_embeddings
    

### Building the `NER` model using the subclassing API

In [28]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
      super(NERModel, self).__init__()
      self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
      self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
      self.dropout1 = keras.layers.Dropout(0.1)
      self.ff = keras.layers.Dense(ff_dim, activation="relu")
      self.dropout2 = keras.layers.Dropout(0.1)
      self.ff_final = keras.layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
      x = self.embedding_layer(inputs)
      x = self.transformer_block(x)
      x = self.dropout1(x, training=training)
      x = self.ff(x)
      x = self.dropout2(x, training=training)
      x = self.ff_final(x)
      return x

### Loading the `CoNLL 2003` dataset

In [29]:
conll_data = load_dataset("conll2003")

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


  0%|          | 0/3 [00:00<?, ?it/s]

We will export this data to a tab-separated file format which will be easy to read as a `tf.data.Dataset` object.

In [30]:
def export_to_file(export_file_path, data):
    with open(export_file_path, "w") as f:
        for record in data:
            ner_tags = record["ner_tags"]
            tokens = record["tokens"]
            f.write(
                str(len(tokens))
                + "\t"
                + "\t".join(tokens)
                + "\t"
                + "\t".join(map(str, ner_tags))
                + "\n"
            )


if os.path.exists("data") == False:
  os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

### Make the NER label lookup table

NER labels are usually provided in IOB, IOB2 or IOBES formats. 


Note that we start our label numbering from 1 since 0 will be reserved for padding. We have a total of 10 labels: 9 from the NER dataset and one for padding.



In [31]:
def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    return dict(zip(range(0, len(all_labels) + 1), all_labels))


mapping = make_tag_lookup_table()
print(mapping)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}


Get a list of all tokens in the training dataset. This will be used to create the vocabulary.

In [32]:
all_tokens = sum(conll_data["train"]["tokens"], [])
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

# We only take (vocab_size - 2) most commons words from the training data since
# the `StringLookup` class uses 2 additional tokens - one denoting an unknown
# token and another one denoting a masking token
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# The StringLook class will convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(
    vocabulary=vocabulary
)

21009


Create 2 new Dataset objects from the training and validation data

In [33]:
train_data = tf.data.TextLineDataset("./data/conll_train.txt")
val_data = tf.data.TextLineDataset("./data/conll_val.txt")

In [34]:
print(list(train_data.take(1).as_numpy_iterator()))

[b'9\tEU\trejects\tGerman\tcall\tto\tboycott\tBritish\tlamb\t.\t3\t0\t7\t0\t0\t0\t7\t0\t0']


We will be using the following map function to transform the data in the dataset:

In [35]:
def map_record_to_training_data(record):
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    tokens = record[1 : length + 1]
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    return tokens, tags


def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)


# We use `padded_batch` here because each record in the dataset has a
# different length.
batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)

ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)

We will be using a custom loss function that will ignore the loss from padded tokens.

In [36]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


loss = CustomNonPaddingTokenLoss()

In [37]:
ner_model.compile(optimizer="adam", loss=loss)
ner_model.fit(train_dataset, epochs=10)


def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)


# Sample inference using the trained model
sample_input = tokenize_and_convert_to_ids(
    "eu rejects german call to boycott british lamb"
)
sample_input = tf.reshape(sample_input, shape=[1, -1])
print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# eu -> B-ORG, german -> B-MISC, british -> B-MISC
print(prediction)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


439/439 [==============================] - 13s 19ms/step - loss: 0.6409
Epoch 2/10
439/439 [==============================] - 8s 19ms/step - loss: 0.2478
Epoch 3/10
439/439 [==============================] - 8s 19ms/step - loss: 0.1531
Epoch 4/10
439/439 [==============================] - 9s 19ms/step - loss: 0.1202
Epoch 5/10
439/439 [==============================] - 8s 19ms/step - loss: 0.0959
Epoch 6/10
439/439 [==============================] - 8s 19ms/step - loss: 0.0782
Epoch 7/10
439/439 [==============================] - 8s 19ms/step - loss: 0.0654
Epoch 8/10
439/439 [==============================] - 9s 19ms/step - loss: 0.0560
Epoch 9/10
439/439 [==============================] - 9s 20ms/step - loss: 0.0496
Epoch 10/10
439/439 [==============================] - 9s 19ms/step - loss: 0.0436
tf.Tensor([[  988 10950   204   628     6  3938   215  5773]], shape=(1, 8), dtype=int64)
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O']


### Metrics calculation
Here is a function to calculate the metrics. The function calculates F1 score for the overall NER dataset as well as individual scores for each NER tag.

In [38]:
def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []

    for x, y in dataset:
        output = ner_model.predict(x)
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]

    evaluate(real_tags, predicted_tags)


calculate_metrics(val_dataset)

processed 51362 tokens with 5942 phrases; found: 5594 phrases; correct: 4015.
accuracy:  65.16%; (non-O)
accuracy:  93.42%; precision:  71.77%; recall:  67.57%; FB1:  69.61
              LOC: precision:  84.35%; recall:  78.61%; FB1:  81.38  1712
             MISC: precision:  72.65%; recall:  65.40%; FB1:  68.84  830
              ORG: precision:  60.55%; recall:  63.53%; FB1:  62.01  1407
              PER: precision:  67.84%; recall:  60.59%; FB1:  64.01  1645


### Conclusions
In this exercise, we created a simple transformer based named entity recognition model. We trained it on the CoNLL 2003 shared task data and got an overall F1 score of around 70%. State of the art NER models fine-tuned on pretrained models such as BERT or ELECTRA can easily get much higher F1 score -between 90-95% on this dataset owing to the inherent knowledge of words as part of the pretraining process and the usage of subword tokenization.


### Copied from:

https://keras.io/examples/nlp/ner_transformers/